In [15]:
import pandas as pd
from forex_python.converter import CurrencyRates
from datetime import datetime
c = CurrencyRates()
# get rates for all unique dates, added tqdm progress bar to see progress
from tqdm import tqdm  
# Change EUR to get any other rate
rates_dict = pd.DataFrame({date: CurrencyRates().get_rates('EUR', date_obj=pd.to_datetime(date, unit='D'))
              for date in tqdm(df['date_order'].unique())})
rates_rub = rates_dict.T
# Change RUB for any other rate conversion with EUR
rates_rub = rates_rub.loc[:,['RUB']]
rates_rub.index.name = 'date_order'

# Use XR to set cost to 1/(RUB to currency_x rate), except when currency is RUB and when servicecost is 0, in those cases just use servicecost
#df['XR'] = df.apply(lambda row: 1.0/rates_dict[row['date_order']][row['rate']]*row['ci=ost'] if row['rate']!='RUB' and row['cost'] != 0 else row['cost'], axis=1)

100%|██████████| 12/12 [00:05<00:00,  2.27it/s]


In [17]:
df = df.merge(rates_rub,on='date_order')
df.rename(columns = {'RUB': 'XR'}, inplace = True)
# Replace XR of RUB with 1
df.loc[(df['Country'] =='Russian Federation'),'XR'] = 1
df['cost_RUB'] = df.cost * df.XR